In [1]:
import napari
from pathlib import Path
from abbott.h5_files import *
import numpy as np
import imageio.v2 as io
import nest_asyncio
nest_asyncio.apply()

In [2]:
fn = Path(r"Q:\carspi\PixelClassTest\ETiX-Cer1-SBR1_s1.h5")

fld = fn.parent

print(h5_summary(fn))

filename:	ETiX-Cer1-SBR1_s1.h5
condition:	NA

  name          shape  dtype   size element_size_um img_type stain cycle wavelength level
/ch_00 (23, 512, 512) uint16  12.06 [3.0, 2.6, 2.6]       NA    NA    NA         NA    NA
/ch_01 (23, 512, 512) uint16  12.06 [3.0, 2.6, 2.6]       NA    NA    NA         NA    NA
/ch_02 (23, 512, 512) uint16  12.06 [3.0, 2.6, 2.6]       NA    NA    NA         NA    NA




In [3]:
from random import randrange

out_fld_raw = fld.parent / (fld.name + '_raw')
out_fld_raw.mkdir(exist_ok=True)

def generate_trainingset_folder(fld, n=100):
    """
    Generate a random set of `n` training images from a folder containing abbott.hdf5 files. Make sure to use a large enough `n` to get a representative sample of your data.
    
    parameters
    ----------
    fld: str or pathlib.Path
        folder containing your input data.
    n: int
        number of training volumes to generate.
    """
    fld = Path(fld)
    for fn in sorted(np.random.choice(list(fld.glob('*.h5')), n, replace=False)):
        print("processing {}...".format(fn.name))
        generate_trainingset(fn)

         
def generate_trainingset(fn):
    fn = Path(fn)
    output_fn = Path(out_fld_raw / (fn.stem + '.tif'))
    
    with h5py.File(fn, 'r') as f:
        channel = f['ch_00'][...]
        channel = channel[randrange(channel.shape[0])] # select a random timepoint

    write_tif_file(output_fn, channel)

def write_tif_file(output_fn, channel):
    print('writing {}...'.format(output_fn.name))
    output_fn.parent.mkdir(exist_ok=True, parents=True)
    io.imwrite(output_fn, channel)
    return output_fn

In [4]:
generate_trainingset_folder(fld, n= 100) # please keep n = 100 so that we have a sufficiently large training / test dataset for convolutional neural net training 

processing ETiX-Cer1-SBR1_s100.h5...
writing ETiX-Cer1-SBR1_s100.tif...
processing ETiX-Cer1-SBR1_s101.h5...
writing ETiX-Cer1-SBR1_s101.tif...
processing ETiX-Cer1-SBR1_s102.h5...
writing ETiX-Cer1-SBR1_s102.tif...
processing ETiX-Cer1-SBR1_s108.h5...
writing ETiX-Cer1-SBR1_s108.tif...
processing ETiX-Cer1-SBR1_s116.h5...
writing ETiX-Cer1-SBR1_s116.tif...
processing ETiX-Cer1-SBR1_s119.h5...
writing ETiX-Cer1-SBR1_s119.tif...
processing ETiX-Cer1-SBR1_s125.h5...
writing ETiX-Cer1-SBR1_s125.tif...
processing ETiX-Cer1-SBR1_s127.h5...
writing ETiX-Cer1-SBR1_s127.tif...
processing ETiX-Cer1-SBR1_s13.h5...
writing ETiX-Cer1-SBR1_s13.tif...
processing ETiX-Cer1-SBR1_s134.h5...
writing ETiX-Cer1-SBR1_s134.tif...
processing ETiX-Cer1-SBR1_s144.h5...
writing ETiX-Cer1-SBR1_s144.tif...
processing ETiX-Cer1-SBR1_s146.h5...
writing ETiX-Cer1-SBR1_s146.tif...
processing ETiX-Cer1-SBR1_s149.h5...
writing ETiX-Cer1-SBR1_s149.tif...
processing ETiX-Cer1-SBR1_s15.h5...
writing ETiX-Cer1-SBR1_s15.tif

In [5]:
# function that opens the training images generated in the previous steps and automatically saves manually automated segmentation mask when viewer is closed

def manual_mask_annotation(fn):
    
    viewer = napari.Viewer()
    print(fn.name)
    file = io.imread(fn)
    viewer.add_image(file, scale=scale)
    lbl_layer = np.zeros(file.shape, dtype=np.uint8)
    viewer.add_labels(lbl_layer, scale=scale)
    viewer.show(block=True)
    # after the viewer is closed, the following will be executed:
    lbl = viewer.layers['lbl_layer'].data
    output_fn = Path(out_fld_mask / (fn.stem + '.tif'))
    io.imwrite(output_fn, lbl)

In [6]:
scale=[2.6, 2.6]
fns = list(Path(out_fld_raw).glob('*.tif'))
out_fld_mask = fld.parent / (fld.name + '_mask')
out_fld_mask.mkdir(exist_ok=True)
    
for fn in fns:
    manual_mask_annotation(fn)

ETiX-Cer1-SBR1_s100.tif
ETiX-Cer1-SBR1_s101.tif
ETiX-Cer1-SBR1_s102.tif
ETiX-Cer1-SBR1_s108.tif
ETiX-Cer1-SBR1_s116.tif
ETiX-Cer1-SBR1_s119.tif
ETiX-Cer1-SBR1_s125.tif
ETiX-Cer1-SBR1_s127.tif
ETiX-Cer1-SBR1_s13.tif
ETiX-Cer1-SBR1_s134.tif
ETiX-Cer1-SBR1_s144.tif
ETiX-Cer1-SBR1_s146.tif
ETiX-Cer1-SBR1_s149.tif
ETiX-Cer1-SBR1_s15.tif
ETiX-Cer1-SBR1_s151.tif
ETiX-Cer1-SBR1_s152.tif
ETiX-Cer1-SBR1_s154.tif
ETiX-Cer1-SBR1_s158.tif
ETiX-Cer1-SBR1_s16.tif
ETiX-Cer1-SBR1_s160.tif
ETiX-Cer1-SBR1_s161.tif
ETiX-Cer1-SBR1_s164.tif
ETiX-Cer1-SBR1_s17.tif
ETiX-Cer1-SBR1_s173.tif
ETiX-Cer1-SBR1_s175.tif
ETiX-Cer1-SBR1_s177.tif
ETiX-Cer1-SBR1_s182.tif
ETiX-Cer1-SBR1_s183.tif
ETiX-Cer1-SBR1_s184.tif
ETiX-Cer1-SBR1_s194.tif
ETiX-Cer1-SBR1_s196.tif
ETiX-Cer1-SBR1_s197.tif
ETiX-Cer1-SBR1_s199.tif
ETiX-Cer1-SBR1_s205.tif
ETiX-Cer1-SBR1_s206.tif
ETiX-Cer1-SBR1_s210.tif
ETiX-Cer1-SBR1_s215.tif
ETiX-Cer1-SBR1_s218.tif
ETiX-Cer1-SBR1_s225.tif
ETiX-Cer1-SBR1_s226.tif
ETiX-Cer1-SBR1_s229.tif
ETiX-Cer1-SBR1_s230.